### Problem Statement:

What is the electrical consumption of an individual household? What times are consumption at its peak and its minimal? 

### Data: 
The data contains measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years. 

**source:** https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

### Data Preperation:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

